Clone git repository

In [ ]:
!git clone https://github.com/umarmuaz/German-News-Data-Classification.git

Cloning into 'German-News-Data-Classification'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 1), reused 15 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 39.78 KiB | 4.97 MiB/s, done.
Resolving deltas: 100% (1/1), done.


Move to the directory

In [ ]:
%cd German-News-Data-Classification/

/content/German-News-Data-Classification


In [ ]:
!pwd

/content/German-News-Data-Classification


Install requirements

In [ ]:
!pip install -r '/content/German-News-Data-Classification/requirements.txt'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Make directories for the data or models

In [ ]:
!mkdir /content/German-News-Data-Classification/data

In [ ]:
!mkdir /content/German-News-Data-Classification/models

Copy the data and models to these folders

In [ ]:
!cp /content/drive/MyDrive/DAT620/new_data_updated.csv /content/German-News-Data-Classification/data/new_data_updated.csv

In [ ]:
!cp /content/drive/MyDrive/DAT620/data.csv /content/German-News-Data-Classification/data/data.csv

In [ ]:
!cp /content/drive/MyDrive/DAT620/Categories_manual_adapted.xlsx /content/German-News-Data-Classification/data/Categories_manual_adapted.xlsx

In [ ]:
!cp -r /content/drive/MyDrive/DAT620/german_finetuned_model /content/German-News-Data-Classification/models/german_finetuned_model

Train the classical machine learning models

In [ ]:
!python -m src.train_classical_models

LogisticRegression Accuracy: 0.7659751037344399
RandomForest Accuracy: 0.7713692946058092
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:20:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
XGBoost Accuracy: 0.7634854771784232
KNN Accuracy: 0.7203319502074689
SVM Accuracy: 0.8120331950207469


Train the transformer models

In [ ]:
!python -m src.train_transformer_models

Evaluate the models

In [ ]:
!python -m src.evaluate_models

LogisticRegression Accuracy: 0.7659751037344399
RandomForest Accuracy: 0.7713692946058092
XGBoost Accuracy: 0.7634854771784232
KNN Accuracy: 0.7203319502074689
SVM Accuracy: 0.8120331950207469
Map: 100% 2410/2410 [00:00<00:00, 249994.38 examples/s]
Map: 100% 2410/2410 [00:00<00:00, 3530.26 examples/s]
2024-12-18 14:46:25.737931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 14:46:25.763812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 14:46:25.772075: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 14:46:25.802311: I tensorflow/core/platform/cpu_fea

Predict category using the best performing model (DistilBERT)

In [ ]:
import torch
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification

le = joblib.load("./models/label_encoder.joblib")
model_path = "./models/german_finetuned_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

texts = ["Krieg in der Ukraine entscheidend Offensive Putins perfider Plan Putin setzen darauf dass RuÃŸland der Ukraine mit noch mehr Soldat bezwingen kÃ¶nnen geht sein Kriegsplan auf oder kommen der Leopard Spitze|Panzer gerade noch rechtzeitig"]
inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predictions = torch.argmax(logits, dim=1).numpy()
predicted_labels = le.inverse_transform(predictions)
print(predicted_labels)

['politik']
